#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [226]:
import pandas as pd
import pymongo
import datetime
import re

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [145]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.0
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.0


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [10]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

Tout d'abord nous regardons les types des features pour s'assurer qu'ils sont cohérents :

In [4]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

Il y a des conversions à faire : backers, goals, pledged, usd pledged en float et deadline et launched en datetime.

- Goal :

Il y a visiblement une erreur lors de l'entrée des données dans goal (sans doute échange de 2 colonnes):

In [40]:
for i, item in enumerate(df_ks['goal']):
   try:
      float(item)
   except ValueError:
      print('ERROR at index {}: {!r}'.format(i, item))

ERROR at index 66141: '2014-04-17'


On peut donc logiquement s'attendre à avoir un problème sur 1 ligne pour tous les features, pour le vérifier nous allons afficher la ligne en question (index 66141) :

In [41]:
df_ks.loc[0, :]

ID                                                        872782264
name                Scott Cooper's Solo CD "A Leg Trick" (Canceled)
category                                                       Rock
main_category                                                 Music
currency                                                        USD
deadline                                                 2011-09-16
goal                                                           2000
launched                                        2011-08-17 06:31:31
pledged                                                        1145
state                                                      canceled
backers                                                          24
country                                                          US
usd pledged                                                    1145
usd_pledged_real                                               1145
Name: 0, dtype: object

In [39]:
df_ks.loc[66141, :]

ID                                        85964225
name                Debut Album from Michael Adam 
category                          Grace is Leaving
main_category                           Indie Rock
currency                                     Music
deadline                                       USD
goal                                    2014-04-17
launched                                     700.0
pledged                        2014-04-02 21:56:35
state                                        850.0
backers                                 successful
country                                         32
usd pledged                                     US
usd_pledged_real                               850
Name: 66141, dtype: object

Cette ligne a bien été mal remplie : visiblement le nom name a été par erreur séparé en 2 (nom = name + category), nous allons la restaurer pour ne pas perdre d'informations :

In [146]:
dic = [85964225, 'Debut Album from Michael Adam Grace is Leaving', 'Indie Rock', 'Music', 'USD', '2014-04-17', 700.0, '2014-04-02 21:56:35', 850.0, "successful", 32, 'US', 850, 850.00]
df_ks.loc[[66141], :] = dic
df_ks.loc[[66141], :]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
66141,85964225,Debut Album from Michael Adam Grace is Leaving,Indie Rock,Music,USD,2014-04-17,700,2014-04-02 21:56:35,850,successful,32,US,850,850.0


Vérifions qu'il n'y a plus de problème à présent:

In [147]:
df_ks.isna().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

Nous allons conserver les lignes pour lesquelles il n'y a pas de NA afin de ne pas perdre d'informations :

In [148]:
df_ks = df_ks[df_ks.isna().any(axis=1)==False]

Nous n'avons plus de NA à présent :

In [140]:
df_ks.isna().sum().sum()

0

Nous pouvons à présent convertir les données sans problème normalement. 

In [149]:
dtype = {'ID' : 'int64', 'name' : 'string', 'category' : 'string', 'main_category' : 'string', 'currency' : 'string', 'deadline' : 'datetime64[ns]',
       'goal' : 'float64', 'launched' : 'datetime64[ns]', 'pledged' : 'float64', 'state' : 'string', 'backers' : 'float64', 'country' : 'string',
       'usd pledged' : 'float64', 'usd_pledged_real' : 'float64'}
df_ks= df_ks.astype(dtype)

La conversion s'est bien déroulée :

In [107]:
df_ks.dtypes

ID                           int64
name                        string
category                    string
main_category               string
currency                    string
deadline            datetime64[ns]
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       string
backers                    float64
country                     string
usd pledged                float64
usd_pledged_real           float64
dtype: object

Le dataframe obtenu contient donc les colonnes :

In [84]:
df_ks.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real'],
      dtype='object')

Afin d'avoir un dataframe plus clair et optimisé (et de réduire les temps d'exécution des requêtes) nous allons passer en revue les questions de cet exercice afin de retirer les colonnes dont nous ne nous servirons visiblement pas :

In [150]:
df_ks = df_ks.drop(['currency', 'deadline', 'usd pledged', 'pledged', 'backers'], axis=1)
df_ks.head(2)

,ID,name,category,main_category,goal,launched,state,country,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,2000.0,2011-08-17 06:31:31,canceled,US,1145.0
1,1326492673,Ohceola jewelry,Fashion,Fashion,18000.0,2012-07-23 20:46:48,failed,US,1851.0


Nous allons prendre l'ID spécifié dans le dataset comme ID dans la base de données Mongo, pour qu'il soit interprété comme tel il faut intégrer une colonne _id :

In [241]:
df_ks['_id'] = df_ks['ID']

Nous allons à présent pouvoir nourrrir la base de données :

In [242]:
DOCUMENTS = df_ks.to_dict('records')

In [243]:
DOCUMENTS[0]

{'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'goal': 2000.0,
 'launched': Timestamp('2011-08-17 06:31:31'),
 'state': 'canceled',
 'country': 'US',
 'usd_pledged_real': 1145.0,
 '_id': 872782264}

Nous insérons en nous assurant qu'il n'y a pas de données dans la base :

In [244]:
collection.drop()

In [245]:
collection.insert_many(DOCUMENTS)

## Question 1  

In [179]:
cur = collection.find({},
 {'name':1}).sort([
    ("usd pledged", 1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fc38a3139e2ae77eff46b8a'),
  'name': 'SketchPlanner: Create and Plan- all in one beautiful book!'},
 {'_id': ObjectId('5fc38a3139e2ae77eff46b8b'),
  'name': 'Proven sales with custom motorcycle accessories'},
 {'_id': ObjectId('5fc38a3139e2ae77eff46b88'), 'name': 'Ohceola jewelry'},
 {'_id': ObjectId('5fc38a3139e2ae77eff46b87'),
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)'},
 {'_id': ObjectId('5fc38a3139e2ae77eff46b89'),
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"'}]

## Question 2

Nous comptons le nombre de projets affichant joyeusement successful dans la colonne state :

In [225]:
cur = collection.find({
     "state": 'successful' 
          })
len(list(cur))

52999

## Question 3

In [193]:
cur = collection.aggregate([{
    "$group" : {
        "_id" : "$main_category", 
        "NumberProjectByCategory" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Theater', 'NumberProjectByCategory': 4305},
 {'_id': 'Journalism', 'NumberProjectByCategory': 1807},
 {'_id': 'Photography', 'NumberProjectByCategory': 4176},
 {'_id': 'Technology', 'NumberProjectByCategory': 12871},
 {'_id': 'Food', 'NumberProjectByCategory': 9805},
 {'_id': 'Comics', 'NumberProjectByCategory': 4139},
 {'_id': 'Art', 'NumberProjectByCategory': 11260},
 {'_id': 'Games', 'NumberProjectByCategory': 13987},
 {'_id': 'Publishing', 'NumberProjectByCategory': 15721},
 {'_id': 'Film & Video', 'NumberProjectByCategory': 24982},
 {'_id': 'Fashion', 'NumberProjectByCategory': 8943},
 {'_id': 'Dance', 'NumberProjectByCategory': 1488},
 {'_id': 'Music', 'NumberProjectByCategory': 19658},
 {'_id': 'Crafts', 'NumberProjectByCategory': 3463},
 {'_id': 'Design', 'NumberProjectByCategory': 11911}]

## Question 4

Par curiosité, regardons combien de projets français sont présents dans le dataset :

In [224]:
cur = collection.find({
     "country": 'FR' 
          })
len(list(cur))

1173

Et avant 2016 il y en a :

In [222]:
cur = collection.find({
     "country": 'FR' 
          ,
          'launched' : {'$lt' : datetime.datetime(2016, 1, 1)}})
len(list(cur))

330

## Question 5

In [223]:
cur = collection.find({
     "country": 'US' 
          ,
          'goal' : {'$gt' : 200000}})
len(list(cur))

1841

## Question 6 

In [237]:
collection.drop_indexes()

In [238]:
collection.create_index([('name', 'text')])

'name_text'

Nous pouvons à présent rechercher le nombre de projet incluant Sport dans leur titre :

In [240]:
cur = collection.find( { "$text": { "$search": "Sport" } } )
len(list(cur))

316